<a href="https://colab.research.google.com/github/Rutijana/Tryout_Rutijana/blob/master/Poker_Sequence_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from dataclasses import dataclass
from typing import List, Tuple

BASE_POT = 60        # 3 players × 20 chips each
BET_SIZE = 10
RAISE_SIZE = 20


@dataclass(frozen=True)
class GameState:
    """Immutable game state for the flop betting round."""
    current_player: int
    actions: str
    contributions: Tuple[int, int, int]
    active: Tuple[bool, bool, bool]
    highest_bet: int
    can_raise: bool

    def is_over(self) -> bool:
        """Determine whether the betting round is finished.

        End conditions:
        - Only one player remains active.
        - Everyone checked (x:x:x) when no bet happened.
        - A bet happened and all active players matched the highest bet.
        """
        alive = [i for i, a in enumerate(self.active) if a]

        # If only one player remains, round ends.
        if len(alive) <= 1:
            return True

        # If no bet was made and everyone checked (three actions happened),
        # end the round (x:x:x).
        if self.highest_bet == 0 and self.actions.count(":") == 2:
            return True

        # If a bet/raise happened, the round ends when all active players
        # have matched the highest bet.
        if self.highest_bet > 0:
            if all(self.contributions[i] == self.highest_bet for i in alive):
                return True

        return False


def get_valid_actions(state: GameState) -> List[str]:
    """Return valid actions in the required exploration order.

    Order chosen to match the expected output:
    - If there's a call amount and raises are allowed: ['r', 'c', 'f'].
    - If there's a call amount and raises not allowed: ['c', 'f'].
    - If no call amount: ['x', 'b'] (bet only if no bettor yet).
    """
    p = state.current_player
    if not state.active[p] or state.is_over():
        return []

    to_call = state.highest_bet - state.contributions[p]
    actions: List[str] = []

    if to_call > 0:
        # Facing a bet
        if state.can_raise:
            actions.extend(["r", "c", "f"])
        else:
            actions.extend(["c", "f"])
    else:
        # No outstanding bet: check first, bet allowed only if no bettor yet
        actions.extend(["x"])
        if state.highest_bet == 0:
            actions.append("b")

    return actions


def apply_action(state: GameState, action: str) -> GameState:
    """Apply an action and return the new state.

    - 'x' : check (no chips)
    - 'b' : bet (adds BET_SIZE)
    - 'c' : call (adds to_call)
    - 'r' : raise (sets everyone's target to RAISE_SIZE; raiser contribution becomes RAISE_SIZE)
    - 'f' : fold (player becomes inactive)
    """
    p = state.current_player
    contrib = list(state.contributions)
    active = list(state.active)
    highest = state.highest_bet
    can_raise = state.can_raise

    to_call = highest - contrib[p]

    if action == "f":
        active[p] = False

    elif action == "x":
        # check: nothing changes for contributions
        pass

    elif action == "b":
        # bet: player posts BET_SIZE
        contrib[p] += BET_SIZE
        highest = BET_SIZE
        can_raise = True

    elif action == "c":
        # call: player posts exactly the to_call
        contrib[p] += to_call

    elif action == "r":
        # RAISE (interpreted as setting the total required bet to RAISE_SIZE).
        # Raiser contribution becomes RAISE_SIZE (not call+raise_size).
        # highest becomes RAISE_SIZE.
        # No further raises allowed.
        contrib[p] = RAISE_SIZE
        highest = RAISE_SIZE
        can_raise = False

    else:
        raise ValueError(f"Unknown action: {action!r}")

    # Advance to next active player (wrap-around)
    nxt = (p + 1) % 3
    while not active[nxt] and nxt != p:
        nxt = (nxt + 1) % 3

    # Format action string tokens with amounts where needed
    if action == "b":
        token = f"b{BET_SIZE}"
    elif action == "r":
        token = f"r{RAISE_SIZE}"
    else:
        token = action

    new_actions = state.actions + (":" if state.actions else "") + token

    return GameState(
        current_player=nxt,
        actions=new_actions,
        contributions=tuple(contrib),
        active=tuple(active),
        highest_bet=highest,
        can_raise=can_raise,
    )


def generate_sequences() -> List[str]:
    """Generate all valid sequences using DFS with the required ordering."""
    start = GameState(
        current_player=0,
        actions="",
        contributions=(0, 0, 0),
        active=(True, True, True),
        highest_bet=0,
        can_raise=True,
    )

    results: List[str] = []

    def dfs(state: GameState) -> None:
        if state.is_over():
            pot = BASE_POT + sum(state.contributions)
            results.append(f"{state.actions}, pot={pot}")
            return

        for act in get_valid_actions(state):
            dfs(apply_action(state, act))

    dfs(start)
    return results


def main() -> None:
    sequences = generate_sequences()
    for seq in sequences:
        print(seq)


if __name__ == "__main__":
    main()


x:x:x, pot=60
x:x:b10:r20:c:c, pot=120
x:x:b10:r20:c:f, pot=110
x:x:b10:r20:f:c, pot=100
x:x:b10:r20:f:f, pot=90
x:x:b10:c:r20:c:c, pot=120
x:x:b10:c:r20:c:f, pot=110
x:x:b10:c:r20:f:c, pot=110
x:x:b10:c:r20:f:f, pot=100
x:x:b10:c:c, pot=90
x:x:b10:c:f, pot=80
x:x:b10:f:r20:c, pot=100
x:x:b10:f:r20:f, pot=90
x:x:b10:f:c, pot=80
x:x:b10:f:f, pot=70
x:b10:r20:c:c, pot=120
x:b10:r20:c:f, pot=110
x:b10:r20:f:c, pot=100
x:b10:r20:f:f, pot=90
x:b10:c:r20:c:c, pot=120
x:b10:c:r20:c:f, pot=110
x:b10:c:r20:f:c, pot=110
x:b10:c:r20:f:f, pot=100
x:b10:c:c, pot=90
x:b10:c:f, pot=80
x:b10:f:r20:c, pot=100
x:b10:f:r20:f, pot=90
x:b10:f:c, pot=80
x:b10:f:f, pot=70
b10:r20:c:c, pot=120
b10:r20:c:f, pot=110
b10:r20:f:c, pot=100
b10:r20:f:f, pot=90
b10:c:r20:c:c, pot=120
b10:c:r20:c:f, pot=110
b10:c:r20:f:c, pot=110
b10:c:r20:f:f, pot=100
b10:c:c, pot=90
b10:c:f, pot=80
b10:f:r20:c, pot=100
b10:f:r20:f, pot=90
b10:f:c, pot=80
b10:f:f, pot=70
